1. celestrak에서 위성번호 가져오고
2. space-track에서 tle 가져와서 공분산 계산
3. 확률 계산 </br>
TCA, HBR은 알고 있다는 전제하에 계산됨.

In [1]:
from sgp4.api import Satrec
from sgp4.api import jday
from astropy.time import Time
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord, TEME
from datetime import timedelta, datetime
import pandas as pd
import requests
import json
import configparser
import requests

## 함수정의

In [13]:
#string을 datetime으로 변환
def str_to_datetime(date_str):

    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
# TLE데이터를 통해 datetime시점에서의 statevector 계산
def get_state_ECI(epoch_target, TLE_Line1, TLE_Line2,i):
    target_day=epoch_target
    firstLine=TLE_Line1
    secondLine=TLE_Line2

    yr, mon, day = target_day.year, target_day.month, target_day.day
    hour, min, sec = target_day.hour, target_day.minute, target_day.second
    jd, fr = jday(yr, mon, day, hour, min, sec)

    satellite = Satrec.twoline2rv(firstLine[i],secondLine[i]) 
    e, r, v = satellite.sgp4(jd,fr)
    r_np=np.array(r)
    v_np=np.array(v)
    return r_np, v_np
# spack-track TLE 주소를 넣고, 특정 feature 불러오기
def get_tle_element(feature,requestFindObjects):
    param=feature
    r_f_d = requestFindObjects
    # set of satellite ids of objects included in TLE data
    SATNO_LIST = []

    # Use configparser package to pull in the ini file (pip install configparser)
    config = configparser.ConfigParser()
    config.read("./Login.ini")
    configUsr = config.get("configuration", "username")
    configPwd = config.get("configuration", "password")
    siteCred = {'identity': configUsr, 'password': configPwd}

    with requests.Session() as session:
        # run the session in a with block to force session to close if we exit

        # need to log in first. note that we get a 200 to say the web site got the data, not that we are logged in
        response = session.post(uriBase + requestLogin, data=siteCred)
        if response.status_code != 200:
            print("Error, POST fail on login")

        # this query picks up objects from the catalog. Note - a 401 failure shows you have bad credentials
        response = session.get(uriBase + requestCmdAction + r_f_d)
        if response.status_code != 200:
            print("Error, GET fail on request")

        data = json.loads(response.text)
        for object in data:
            SATNO = object[f'{feature}']
            SATNO_LIST.append(SATNO)
    
        session.close()
        SATNO_LIST=np.array(SATNO_LIST).reshape(-1,1)
    return SATNO_LIST 

'''
세줄짜리 TLE 텍스트를 input
space-track에서 tle를 가져오고 공분산 계산
'''
def covariance(Object_TLE):
    obj = Object_TLE
    objnum = int(obj[1][2:7])


    startdate = "2023-08-01" #시작일
    enddate = str(datetime.now()) #종료일
    requestFindObjects = r= f"/class/tle/EPOCH/>{startdate}%2C<{enddate}/NORAD_CAT_ID/{objnum}/orderby/EPOCH%20asc"

    # 필요한 feature 불러오기
    EPOCH_total = get_tle_element("EPOCH",requestFindObjects)
    TLE_LINE1_total = get_tle_element("TLE_LINE1",requestFindObjects)
    TLE_LINE2_total = get_tle_element("TLE_LINE2",requestFindObjects)
    TLE_total = np.concatenate((EPOCH_total, TLE_LINE1_total, TLE_LINE2_total), axis=1)


    epoch_total = [arr[0] for arr in TLE_total]
    Line1_total = [arr[1] for arr in TLE_total]
    Line2_total = [arr[2] for arr in TLE_total]

    epoch_date_total = np.vectorize(str_to_datetime)(epoch_total) #datetime으로 변환.

    # most recent 시점 찾기
    jul_date = obj[1][18:32] # julian date
    current_date_idx = 0 # Celestrak 충돌확률의 INPUT TLE의 위치.

    for i in range(len(Line1_total)):
        if Line1_total[i][18:32] == jul_date: #"2023-09-05 00:53:04"
            current_date_idx = i
            break

    # 14 일 치 다시 받아오기
    delta = timedelta(days=14)
    startdate = str(epoch_date_total[current_date_idx]-delta)
    enddate = str(epoch_date_total[current_date_idx+1]) #종료일
    requestFindObjects_new = r= f"/class/tle/EPOCH/>{startdate}%2C<{enddate}/NORAD_CAT_ID/{objnum}/orderby/EPOCH%20asc"

    EPOCH = get_tle_element("EPOCH",requestFindObjects_new)
    TLE_LINE1 = get_tle_element("TLE_LINE1",requestFindObjects_new)
    TLE_LINE2 = get_tle_element("TLE_LINE2",requestFindObjects_new)
    TLE=np.concatenate((EPOCH,TLE_LINE1, TLE_LINE2), axis=1)


    epoch = [arr[0] for arr in TLE]
    Line1 = [arr[1] for arr in TLE]
    Line2 = [arr[2] for arr in TLE]

    epoch_date = np.vectorize(str_to_datetime)(epoch) #datetime으로 변환.
    
    # 잔차구하기.
    epoch_valid=epoch_date
    most_recent_epoch = epoch_valid[-1] # 가장 최근시점을 예측한다.
    N = epoch_valid.size #총 TLE 개수 24개 

    delta_r = []
    delta_v = []

    true_position, true_velocity = get_state_ECI(most_recent_epoch, Line1, Line2, N-1)

    for i in range(N-1): # 기준 시점은 제외.
        r_ECI, v_ECI = get_state_ECI(most_recent_epoch, Line1, Line2,i)
        r_residual = r_ECI-true_position
        v_residual = v_ECI-true_velocity

        r_residual = tuple(r_residual)
        v_residual = tuple(v_residual)
        
        delta_r.append((r_residual)) #잔차 r
        delta_v.append((v_residual)) #잔차 v
    delta_r_np = np.array(delta_r) # 23 X 3
    delta_v_np = np.array(delta_v) # 23 X 3
    bar_X_epoch = np.hstack((delta_r_np, delta_v_np)) # 23 X 6

    # 평균 계산
    m = np.mean(bar_X_epoch,axis=0) # 1 X 6

    a=bar_X_epoch-m

    cov = np.zeros((6, 6))

    for i in range(len(a)):
        delta_X = np.array(a[i]).reshape(6, 1) # 6 X 1
        delta_X_T = np.transpose(delta_X) # 1 X 6
        temp = delta_X@delta_X_T # 6 X 6
        cov += temp
    return cov/len(a)

## Celestrak 에서 구하고 싶은 두 위성의 TLE 가져오기

STARLINK-5626 VS SL-8 DEB 

In [3]:
url ="https://celestrak.org/SOCRATES/data.php?CATNR=55435,6396" # Celestrak tle ( 2개 )

r = requests.get(url)
TLE_data=r.text
TLE_split=TLE_data.split('\n')
my_TLE = [line.replace('\r', '') for line in TLE_split]
# TLE 나누기.
Object1_TLE = my_TLE[:3]
Object2_TLE = my_TLE[3:]

## 공분산 계산하기 

In [14]:
# julian_date 예상 범위에 있는 TLE 모두 긁어오기.
uriBase = "https://www.space-track.org"
requestLogin = "/ajaxauth/login"
requestCmdAction = "/basicspacedata/query"

cov1= covariance(Object1_TLE)
cov2 = covariance(Object2_TLE)


# size 관련 정보 불러오기

In [9]:
def get_discos_tle_objects(SATNO_CHUNK_STR):
    response = requests.get(
        f'{DISCOS_URL}/api/objects',
        headers={
            'Authorization': f'Bearer {DISCOS_TOKEN}',
            'DiscosWeb-Api-Version': '2',
        },
        params={
            'filter': f'in(satno,{SATNO_CHUNK_STR})',
            'page[size]': f'{DISCOS_PAGE_SIZE_MAX}',
            'sort': 'satno',            
        }
    )
    doc = response.json()
    DISCOS_OBJECTS = doc['data']
    return DISCOS_OBJECTS

DISCOS_URL = 'https://discosweb.esoc.esa.int'
DISCOS_TOKEN = 'Ijk1YjRjMWYwLTc1ZDItNDc4NS04MzQ4LTgyZmM3MDk2M2IyYiI.lqet3mWHZooxHwUGaRqLBOTvYjI'
DISCOS_SATNO_MAX = 56685
DISCOS_PAGE_SIZE_MAX = 100

objnum1 = int(Object1_TLE[1][2:7])
objnum2 = int(Object2_TLE[1][2:7])
List=[(objnum1),(objnum2)]
obj_dict = dict()
if __name__ == '__main__':
    SATNO_LIST = List
    for idx, chk in enumerate(List): # idx :  인덱스  chk: 값
        
        

        SATNO_CHUNK_STR = '(' + str(chk) + ')'
        print(SATNO_CHUNK_STR)
        DISCOS_OBJECTS = get_discos_tle_objects(SATNO_CHUNK_STR)
        for obj in DISCOS_OBJECTS:
            obj_dict[obj['attributes']['satno']] = obj['attributes']
       

(55435)
(6396)


# Matlab 활용/ 확률 계산

In [8]:
import matlab.engine
eng=matlab.engine.start_matlab()#시작하기
# eng.desktop(nargout=0)# 동기화된 GUI 띄위기

celestrak의 TCA를 이용

In [10]:
TCA = str_to_datetime("2023-09-07 13:08:50")

In [25]:
yr, mon, day = TCA.year, TCA.month, TCA.day
hour, min, sec = TCA.hour, TCA.minute, TCA.second
jd, fr = jday(yr, mon, day, hour, min, sec)

satellite = Satrec.twoline2rv(Object1_TLE[1],Object1_TLE[2]) 
e1, r1, v1 = satellite.sgp4(jd,fr)
satellite = Satrec.twoline2rv(Object2_TLE[1],Object2_TLE[2]) 
e2, r2, v2 = satellite.sgp4(jd,fr)

r1      = matlab.double(list(r1)) 
v1      = matlab.double(list(v1)) 
r2      = matlab.double(list(r2))
v2      = matlab.double(list(v2))
cov1    = matlab.double(list(cov1))
cov2    = matlab.double(list(cov2))
HBR     = 2.22 
Tol     = 1e-09
HBRType = 'squareEquArea' # 영역설정 방법 : 직사각형으로 계산
empty_cov=matlab.double(np.zeros((3,3)))

확률 계산

In [31]:
# matlab 함수 위치 설정
eng.addpath('C:/Users/VDRC/Documents/MATLAB/single_covariance_maximum_Pc/Main/ProbabilityOfCollisionCode',nargout=0)
print("Maximum Pc:",eng.FrisbeeMaxPc(r1,v1,cov1,r2,v2,cov2,HBR,Tol,HBRType)) 
print("Maximum Pc(empty):",eng.FrisbeeMaxPc(r1,v1,cov1,r2,v2,empty_cov,HBR,Tol,HBRType)) 
print("Foster Pc:",eng.Pc2D_Foster(r1,v1,cov1,r2,v2,cov2,HBR,Tol,HBRType)) 

Maximum Pc: 0.0005538545701407567
Maximum Pc(empty): 3.143832451361428e-05
Foster Pc: 2.7060157349009515e-05


예제

In [30]:
r1      = matlab.double([378.39559, 4305.721887, 5752.767554])
v1      = matlab.double([2.360800244, 5.580331936, -4.322349039])
r2      = matlab.double([374.5180598, 4307.560983, 5751.130418])
v2      = matlab.double([-5.388125081, -3.946827739, 3.322820358])
cov1    = matlab.double([[44.5757544811362,  81.6751751052616,  -67.8687662707124],
           [81.6751751052616,  158.453402956163,  -128.616921644857],
           [-67.8687662707124, -128.616921644858, 105.490542562701]])
cov2    = matlab.double([[2.31067077720423,  1.69905293875632  ,-1.4170164577661],
             [1.69905293875632,  1.24957388457206 , -1.04174164279599],
            [-1.4170164577661  ,-1.04174164279599, 0.869260558223714]])
HBR     = 0.020
Tol     = 1e-09
HBRType = 'squareEquArea'

#Pc      = 2.70601573490093e-05
